#Imports and Setup

In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"


#Model loading

In [ ]:
model = models.densenet121(weights="IMAGENET1K_V1")
model.classifier = nn.Linear(model.classifier.in_features, 1)
model.load_state_dict(torch.load("/content/saved_models/best_interpretability_model.pth"))
model = model.to(device)
model.eval()
print("✔ GradCAM++ using DenseNet121 loaded.")


#Detect Last Conv Layer

In [ ]:
def find_last_conv(model):
    layer = None
    for _, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            layer = module
    return layer

target_layer = find_last_conv(model)
print("✔ Target layer:", target_layer)


#Grad-CAM++ Class

In [ ]:
class GradCAMPlusPlus:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        def forward_hook(_, __, out): self.activations = out.detach()
        def backward_hook(_, grad_in, grad_out): self.gradients = grad_out[0].detach()
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

    def generate(self, x):
        self.model.zero_grad()
        out = self.model(x)
        logit = out[:, 0]
        logit.backward(retain_graph=True)
        A = self.activations
        dA = self.gradients
        alpha = dA.pow(2) / (2 * dA.pow(2) + (A * dA.pow(3)).sum(dim=(2,3), keepdim=True) + 1e-9)
        weights = (alpha * dA.relu()).sum(dim=(2,3), keepdim=True)
        cam = (weights * A).sum(dim=1).squeeze()
        cam = torch.relu(cam)
        cam -= cam.min()
        cam /= (cam.max() + 1e-9)
        return cam.cpu().numpy()

gradcampp = GradCAMPlusPlus(model, target_layer)
print("✔ GradCAM++ initialized.")


#Overlay CAM on Image and show Gradcam++

In [ ]:
def overlay_cam(img_path, cam):
    orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    orig = cv2.resize(orig, (224, 224))
    orig_rgb = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)
    cam = cv2.resize(cam, (224, 224))
    heat = cv2.applyColorMap((cam * 255).astype(np.uint8), cv2.COLORMAP_JET)
    overlay = (0.45 * heat + 0.55 * orig_rgb).astype("uint8")
    return overlay[:, :, ::-1]


In [ ]:
def show_gradcam(img_path):
    pil = Image.open(img_path).convert("L")
    x = test_tfms(pil).unsqueeze(0).to(device)
    cam = gradcampp.generate(x)
    overlay = overlay_cam(img_path, cam)
    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.title("GradCAM++ — DenseNet121")
    plt.axis("off")
    plt.show()


#Select Images (3 Bacterial + 5 Viral)

In [ ]:
import os, random

clean_pneu = "/content/chest_xray_cleaned/test/PNEUMONIA"
files = sorted(os.listdir(clean_pneu))
print("Total cleaned pneumonia images:", len(files))

bacteria_imgs = random.sample([f for f in files if "bacteria" in f.lower()], 3)
virus_imgs = random.sample([f for f in files if "virus" in f.lower()], 5)

print("Selected BACTERIAL images:")
for b in bacteria_imgs: print(" -", b)

print("\nSelected VIRAL images:")
for v in virus_imgs: print(" -", v)


In [ ]:
print("\nRunning GradCAM++...\n")
for img_name in bacteria_imgs + virus_imgs:
    path = f"/content/chest_xray_cleaned/test/PNEUMONIA/{img_name}"
    print(f"\n🔍 Image: {img_name}")
    show_gradcam(path)


#Quantitative Metrics: Drop / Increase

In [ ]:
def avg_drop_increase(samples):
    drops, increases = [], []
    model.eval()
    for fp in tqdm(samples, desc="Calculating Drop/Increase"):
        t = get_image_tensor(fp)
        base_p = get_prob_from_model(t)
        cam = gradcampp.generate(t)
        cam = normalize_cam(upsample_cam(cam, (IMG_SIZE, IMG_SIZE)))
        thr = np.percentile(cam, 60)
        mask = (cam >= thr).astype(np.float32)
        mask_t = torch.tensor(mask, dtype=torch.float32).to(device).unsqueeze(0).unsqueeze(0).repeat(1,3,1,1)
        x_masked = t * mask_t
        masked_p = get_prob_from_model(x_masked)
        drop = max(0.0, (base_p - masked_p) / (base_p + 1e-9)) * 100.0
        drops.append(drop)
        increases.append(1.0 if masked_p > base_p else 0.0)
    return np.mean(drops), np.mean(increases) * 100.0


#Quantitative Metrics: Insertion / Deletion AUC

In [ ]:
from sklearn.metrics import auc as calc_auc

def compute_ins_del_auc(samples, steps=100):
    ins_aucs, del_aucs = [], []
    model.eval()
    for fp in tqdm(samples, desc="Calculating Insertion/Deletion AUC"):
        t = get_image_tensor(fp)
        cam = gradcampp.generate(t)
        cam = normalize_cam(upsample_cam(cam, (IMG_SIZE, IMG_SIZE)))
        ins_p = insertion_deletion(fp, cam, steps=steps, mode='insertion')
        del_p = insertion_deletion(fp, cam, steps=steps, mode='deletion')
        ins_aucs.append(calc_auc(np.linspace(0,1,len(ins_p)), ins_p))
        del_aucs.append(calc_auc(np.linspace(0,1,len(del_p)), del_p))
    return np.mean(ins_aucs), np.mean(del_aucs)


#Evaluate (Bacterial + Viral + Normal)

In [ ]:
normal_root = "/content/chest_xray_cleaned/test/NORMAL"
all_normals = sorted([os.path.join(normal_root, f) for f in os.listdir(normal_root) if f.endswith((".jpg",".jpeg",".png"))])
normal_samples = all_normals[:3]

samples_all = [
    "/content/chest_xray_cleaned/test/PNEUMONIA/person122_bacteria_584.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person157_bacteria_739.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person101_bacteria_485.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person43_virus_92.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person1627_virus_2819.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person22_virus_55.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person38_virus_84.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person1672_virus_2888.jpeg",
] + normal_samples

print(f"Prepared {len(samples_all)} images for evaluation.")


#Final Quantitative Scores

In [ ]:
print("\nRunning Quantitative Metrics...")
avg_drop, avg_increase = avg_drop_increase(samples_all)
ins_auc, del_auc = compute_ins_del_auc(samples_all, steps=50)

print("\n" + "="*40)
print(" FINAL AGGREGATE RESULTS (Normal + Pneumonia)")
print("="*40)
print(f"Images Evaluated : {len(samples_all)}")
print(f"Avg Drop (%)     : {avg_drop:.2f}")
print(f"Avg Increase (%) : {avg_increase:.2f}")
print(f"Insertion AUC    : {ins_auc:.4f}")
print(f"Deletion AUC     : {del_auc:.4f}")
print("="*40)


<br> <br> <br> <br>


#BUILDING RISE PIPELINE

In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.densenet121(weights="IMAGENET1K_V1")
model.classifier = nn.Linear(model.classifier.in_features, 1)
model.load_state_dict(torch.load("/content/saved_models/best_interpretability_model.pth"))
model.to(device)
model.eval()

print("✔ Loaded DenseNet121 for RISE.")


#Utility Functions

In [ ]:
def denorm_img(t):
    x = t.cpu().numpy().transpose(1,2,0)
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    x = x * std + mean
    return np.clip(x, 0, 1)

def norm_for_model(x_np):
    mean = np.array([0.485, 0.456, 0.406])[None,None,:]
    std  = np.array([0.229, 0.224, 0.225])[None,None,:]
    x = (x_np - mean) / std
    return torch.tensor(x.transpose(2,0,1)).float().to(device)

def preprocess_np(path):
    pil = Image.open(path).convert("L")
    t = test_tfms(pil).unsqueeze(0).to(device)
    x_np = denorm_img(t[0])
    return x_np, t


#Final RISE Class

In [ ]:
class RISE:
    def __init__(self, model, N=1200, s=16, p=0.5):
        self.model = model
        self.N, self.s, self.p = N, s, p
        self.masks = self.generate_masks().astype("float32")

    def generate_masks(self):
        base = (np.random.rand(self.N, self.s, self.s) < self.p).astype("float32")
        masks = np.zeros((self.N, 224, 224), dtype=np.float32)
        for i in range(self.N):
            m = cv2.resize(base[i], (224,224), interpolation=cv2.INTER_LINEAR)
            masks[i] = cv2.GaussianBlur(m, (11,11), 0)
        return masks

    def explain(self, x_np):
        masks = self.masks
        imgs = [x_np * masks[i][...,None] for i in range(self.N)]
        imgs = torch.stack([norm_for_model(im) for im in imgs], dim=0)

        probs = []
        with torch.no_grad():
            for i in range(0, self.N, 64):
                p = torch.sigmoid(self.model(imgs[i:i+64])).cpu().numpy().flatten()
                probs.extend(p)

        sal = np.tensordot(np.array(probs), masks, axes=(0,0))
        sal -= sal.min()
        sal /= sal.max() + 1e-8
        sal = cv2.GaussianBlur(sal, (21,21), 0)
        return sal

rise = RISE(model, N=1200, s=16, p=0.5)
print("✔ RISE ready.")


#Adaptive Heatmap Overlay

In [ ]:
def overlay_rise_clean(img_path, sal, percentile=75):
    orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if orig is None: return None
    orig = cv2.resize(orig, (224,224))
    orig_rgb = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)

    sal = cv2.GaussianBlur(cv2.resize(sal, (224,224)), (15,15), 0)
    cutoff = np.percentile(sal, percentile)

    mask = sal >= cutoff
    sal_norm = np.maximum(sal - cutoff, 0)
    sal_norm /= sal_norm.max() + 1e-9

    heat = cv2.applyColorMap((sal_norm * 255).astype(np.uint8), cv2.COLORMAP_JET)
    final = orig_rgb.copy()
    overlay = cv2.addWeighted(orig_rgb, 0.45, heat, 0.55, 0)
    final[mask] = overlay[mask]

    return final[:, :, ::-1]


In [ ]:
def show_rise_clean(img_path, label_text):
    pil = Image.open(img_path).convert("L")
    x = test_tfms(pil).unsqueeze(0).to(device)

    with torch.no_grad():
        score = torch.sigmoid(model(x)).item()

    x_np = x[0].permute(1,2,0).cpu().numpy()
    sal = rise.explain(x_np)

    overlay = overlay_rise_clean(img_path, sal, percentile=75)
    if overlay is None: return

    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.title(f"{label_text} — Conf: {score:.4f}")
    plt.axis("off")
    plt.show()


#Running on Selected Images

In [ ]:
paths = [
    "/content/chest_xray_cleaned/test/PNEUMONIA/person122_bacteria_584.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person157_bacteria_739.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person101_bacteria_485.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person43_virus_92.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person1627_virus_2819.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person22_virus_55.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person38_virus_84.jpeg",
    "/content/chest_xray_cleaned/test/PNEUMONIA/person1672_virus_2888.jpeg"
]

print(f"\nProcessing {len(paths)} images...\n")
for p in paths:
    lbl = "BACTERIA" if "bacteria" in p else "VIRUS"
    print(f"→ {lbl}: {p.split('/')[-1]}")
    show_rise_clean(p, lbl)


<br><br><br><br>

In [ ]:

# FULL XAI PIPELINE — SETUP + MODEL + 3 METHODS
import os, cv2, torch, random, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from google.colab import files
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Transforms (same as evaluation stage) ----
IMG_SIZE = 224
test_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ---- Load trained DenseNet121 (Interpretability Model) ----
model = models.densenet121(weights="IMAGENET1K_V1")
model.classifier = nn.Linear(model.classifier.in_features, 1)
model.load_state_dict(torch.load("/content/saved_models/best_interpretability_model.pth", map_location=device))
model.to(device)
model.eval()

print("✔ DenseNet121 loaded for XAI")


#  GRADCAM++ (FINAL VERSION)

class GradCAMPP:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def __call__(self, x):
        logit = self.model(x)
        score = logit[:, 0]
        self.model.zero_grad()
        score.backward(retain_graph=True)

        gradients = self.gradients
        activations = self.activations
        b, k, u, v = gradients.size()

        alpha_num = gradients.pow(2)
        alpha_denom = 2 * gradients.pow(2) + torch.sum(activations * gradients.pow(3), axis=(2, 3)).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alphas = alpha_num / alpha_denom

        weights = torch.maximum(score.exp() * gradients, torch.tensor(0.0).to(gradients.device))
        weights = torch.sum(alphas * weights, axis=(2, 3)).view(b, k, 1, 1)

        saliency_map = torch.sum(weights * activations, axis=1)
        saliency_map = F.relu(saliency_map)
        saliency_map = saliency_map.view(b, -1)
        saliency_map -= saliency_map.min(1, keepdim=True)[0]
        saliency_map /= saliency_map.max(1, keepdim=True)[0]
        saliency_map = saliency_map.view(b, u, v)
        return saliency_map.data

target_layer = model.features.denseblock4.denselayer16.conv2
gradcam_pp = GradCAMPP(model, target_layer)
print("✔ GradCAM++ initialized")
\
#  ABLATIONCAM

def ablation_cam(x):
    out = model.features(x)
    B, C, H, W = out.shape
    base = torch.sigmoid(model(x)).item()
    cam = torch.zeros((H, W), device=x.device)

    for c in range(C):
        masked = out.clone()
        masked[:, c] = 0
        score = torch.sigmoid(model.classifier(masked.mean(dim=(2,3)))).item()
        cam += (base - score)

    cam -= cam.min()
    cam /= (cam.max() + 1e-9)
    return cam.detach().cpu().numpy()

print("✔ AblationCAM initialized")

#  RISE (HIGH-RES ADAPTIVE VERSION)

def denorm_img(t):
    x = t.cpu().numpy().transpose(1,2,0)
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    return np.clip(x * std + mean, 0, 1)

def preprocess_np(path):
    pil = Image.open(path).convert("L")
    t = test_tfms(pil).unsqueeze(0).to(device)
    return denorm_img(t[0]), t

class RISE:
    def __init__(self, model, N=1200, s=16, p=0.5):
        self.model = model
        self.N = N
        self.s = s
        self.p = p
        self.masks = self.generate_masks()

    def generate_masks(self):
        base = (np.random.rand(self.N, self.s, self.s) < self.p).astype("float32")
        masks = np.zeros((self.N, 224, 224), dtype=np.float32)
        for i in range(self.N):
            m = cv2.resize(base[i], (224,224), interpolation=cv2.INTER_LINEAR)
            masks[i] = cv2.GaussianBlur(m, (11,11), 0)
        return masks

    def explain(self, x_np):
        masks = self.masks
        imgs = np.stack([x_np * m[...,None] for m in masks])
        mean = torch.tensor([0.485,0.456,0.406], device=device)[None,:,None,None]
        std  = torch.tensor([0.229,0.224,0.225], device=device)[None,:,None,None]
        imgs = torch.tensor(imgs).permute(0,3,1,2).float().to(device)
        imgs = (imgs - mean) / std

        probs = []
        with torch.no_grad():
            for i in range(0, self.N, 64):
                out = self.model(imgs[i:i+64])
                probs.extend(torch.sigmoid(out).cpu().numpy().flatten())
        sal = np.tensordot(np.array(probs), masks, axes=(0,0))
        sal -= sal.min()
        sal /= (sal.max() + 1e-8)
        sal = cv2.GaussianBlur(sal, (21,21), 0)
        return sal

rise = RISE(model, N=1200, s=16, p=0.5)
print("✔ RISE initialized")


In [ ]:
# RANDOM SAMPLING + OVERLAY + 3×3 PANELS (3 METHODS)

import random
import matplotlib.pyplot as plt

# RANDOM IMAGE SELECTION
normal_dir = "/content/chest_xray_cleaned/test/NORMAL"
pneu_dir   = "/content/chest_xray_cleaned/test/PNEUMONIA"

all_normal   = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith(('.jpeg','.jpg','.png'))]
all_pneu     = [os.path.join(pneu_dir, f) for f in os.listdir(pneu_dir) if f.endswith(('.jpeg','.jpg','.png'))]
all_bacteria = [f for f in all_pneu if "bacteria" in f.lower()]
all_virus    = [f for f in all_pneu if "virus" in f.lower()]

sel_normal   = random.sample(all_normal, 3)
sel_bacteria = random.sample(all_bacteria, 3)
sel_virus    = random.sample(all_virus, 3)

paths  = sel_normal + sel_bacteria + sel_virus
labels = ["Normal 1","Normal 2","Normal 3",
          "Bacteria 1","Bacteria 2","Bacteria 3",
          "Virus 1","Virus 2","Virus 3"]

print(f"Selected {len(paths)} images (3 Normal, 3 Bacteria, 3 Virus)")

# ------------------------- OVERLAY FUNCTION -------------------------
def create_overlay(img_path, heatmap, use_adaptive=False):
    orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    orig = cv2.resize(orig, (224,224))
    orig_rgb = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)
    heatmap = cv2.resize(heatmap, (224,224))

    if use_adaptive:
        heatmap = cv2.GaussianBlur(heatmap, (15, 15), 0)
        cutoff = np.percentile(heatmap, 75)
        mask_indices = heatmap < cutoff

        heatmap_norm = heatmap.copy()
        heatmap_norm -= cutoff
        heatmap_norm[heatmap_norm < 0] = 0
        heatmap_norm /= (heatmap_norm.max() + 1e-9)

        heat_color = cv2.applyColorMap((heatmap_norm * 255).astype(np.uint8), cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(orig_rgb, 0.5, heat_color, 0.5, 0)

        final_img = orig_rgb.copy()
        final_img[~mask_indices] = overlay[~mask_indices]
        final_img[mask_indices]  = orig_rgb[mask_indices]
        return final_img[:, :, ::-1]

    heat_color = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(orig_rgb, 0.5, heat_color, 0.5, 0)
    return overlay[:, :, ::-1]

# ---------------------- GENERATE ALL 3 METHODS ----------------------
rise_images      = []
grad_images      = []
ablation_images  = []

print("Generating RISE + GradCAM++ + AblationCAM maps...")

for p in tqdm(paths):
    pil = Image.open(p).convert("L")
    x = test_tfms(pil).unsqueeze(0).to(device)
    x_np = x[0].permute(1,2,0).cpu().numpy()

    rise_map = rise.explain(x_np)
    rise_images.append(create_overlay(p, rise_map, use_adaptive=True))

    grad_map = gradcam_pp(x).cpu().numpy()[0]
    grad_images.append(create_overlay(p, grad_map, use_adaptive=False))

    abl_map = ablation_cam(x)
    ablation_images.append(create_overlay(p, abl_map, use_adaptive=False))

# --------------------------- PANEL FUNCTION ---------------------------
def plot_3x3_panel(image_list, title_main):
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    fig.suptitle(title_main, fontsize=20, weight="bold", y=0.98)
    row_titles = ["NORMAL", "BACTERIA", "VIRUS"]

    for i, ax in enumerate(axes.flat):
        ax.imshow(image_list[i])
        ax.set_title(labels[i], fontsize=10)
        if i % 3 == 0:
            ax.set_ylabel(row_titles[i//3], fontsize=14, weight="bold")
        ax.set_xticks([]); ax.set_yticks([])
        for spine in ax.spines.values(): spine.set_visible(False)

    plt.tight_layout()
    plt.show()

# ------------------------------ SHOW PANELS ------------------------------
print("\n---------------- RISE PANEL ----------------")
plot_3x3_panel(rise_images, "RISE (Adaptive Threshold — Top 25%)")

print("\n---------------- GRADCAM++ PANEL ----------------")
plot_3x3_panel(grad_images, "GradCAM++ (Gradient-Based)")

print("\n---------------- ABLATIONCAM PANEL ----------------")
plot_3x3_panel(ablation_images, "AblationCAM (Perturbation-Based)")


In [ ]:

#  BLOCK 3 — Quantitative Metrics, Export, Panels, Error Analysis


import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from google.colab import files

print("Running quantitative evaluation...")

# --------------------------- Helper: Clean saliency map ---------------------------
def clean_map(saliency_map, percentile=75):
    if saliency_map.shape != (224,224):
        saliency_map = cv2.resize(saliency_map, (224, 224))
    saliency_map = cv2.GaussianBlur(saliency_map, (15, 15), 0)
    cutoff = np.percentile(saliency_map, percentile)
    saliency_map[saliency_map < cutoff] = 0
    saliency_map -= saliency_map.min()
    saliency_map /= (saliency_map.max() + 1e-9)
    return saliency_map

# ---------------------- Metric 1: Sparsity / Complexity ----------------------
metrics_clean = {"RISE":{"sparsity":[], "entropy":[]},
                 "GradCAM":{"sparsity":[], "entropy":[]},
                 "Ablation":{"sparsity":[], "entropy":[]}}

print("Computing Sparsity and Entropy on CLEAN maps...")
for p in tqdm(paths):
    pil = Image.open(p).convert("L")
    x = test_tfms(pil).unsqueeze(0).to(device)
    x_np = x[0].permute(1,2,0).cpu().numpy()

    rise_raw = rise.explain(x_np)
    grad_raw = gradcam_pp(x).cpu().numpy()[0]
    abl_raw  = ablation_cam(x)

    rise_clean = clean_map(rise_raw)
    grad_clean = clean_map(grad_raw)
    abl_clean  = clean_map(abl_raw)

    metrics_clean["RISE"]["sparsity"].append(calculate_sparsity(rise_clean))
    metrics_clean["RISE"]["entropy"].append(calculate_complexity(rise_clean))

    metrics_clean["GradCAM"]["sparsity"].append(calculate_sparsity(grad_clean))
    metrics_clean["GradCAM"]["entropy"].append(calculate_complexity(grad_clean))

    metrics_clean["Ablation"]["sparsity"].append(calculate_sparsity(abl_clean))
    metrics_clean["Ablation"]["entropy"].append(calculate_complexity(abl_clean))

# ---------------------- Metric 2: Faithfulness (RAW maps) ----------------------
def calculate_faithfulness(model, img_tensor, saliency_map, num_masks=50):
    model.eval()
    if saliency_map.shape != (224,224):
        saliency_map = cv2.resize(saliency_map, (224, 224))
    with torch.no_grad():
        base_pred = torch.sigmoid(model(img_tensor)).item()

    flattened = saliency_map.flatten()
    sorted_idx = np.argsort(flattened)[::-1]
    total_pixels = len(flattened)
    step = max(1, total_pixels // num_masks)

    masked_img = img_tensor.clone()
    intensities_removed = []
    score_drops = []

    for i in range(0, total_pixels, step):
        idx = sorted_idx[i : i+step]
        if len(idx) == 0:
            break
        intensities_removed.append(np.sum(flattened[idx]))
        ys = idx // 224
        xs = idx % 224
        masked_img[0, :, ys, xs] = 0.0
        with torch.no_grad():
            new_pred = torch.sigmoid(model(masked_img)).item()
        score_drops.append(base_pred - new_pred)

    if np.std(score_drops) == 0 or np.std(intensities_removed) == 0:
        return 0
    return pearsonr(intensities_removed, score_drops)[0]

faith_scores = {"RISE":[], "GradCAM":[], "Ablation":[]}

print("Computing Faithfulness Correlation on RAW maps...")
for p in tqdm(paths):
    pil = Image.open(p).convert("L")
    x = test_tfms(pil).unsqueeze(0).to(device)
    x_np = x[0].permute(1,2,0).cpu().numpy()

    rise_raw = rise.explain(x_np)
    grad_raw = gradcam_pp(x).cpu().numpy()[0]
    abl_raw  = ablation_cam(x)

    faith_scores["RISE"].append(calculate_faithfulness(model, x, rise_raw))
    faith_scores["GradCAM"].append(calculate_faithfulness(model, x, grad_raw))
    faith_scores["Ablation"].append(calculate_faithfulness(model, x, abl_raw))

# ---------------------- Building CSV tables ----------------------
print("Saving metric tables...")

df_raw = pd.DataFrame({
    "Method": ["GradCAM++", "AblationCAM", "RISE"],
    "Faithfulness_Correlation": [
        np.mean(faith_scores["GradCAM"]),
        np.mean(faith_scores["Ablation"]),
        np.mean(faith_scores["RISE"]),
    ]
})
df_raw.to_csv("/content/table_raw_metrics.csv", index=False)

df_clean = pd.DataFrame({
    "Method": ["GradCAM++", "AblationCAM", "RISE"],
    "Sparsity_Gini_Clean": [
        np.mean(metrics_clean["GradCAM"]["sparsity"]),
        np.mean(metrics_clean["Ablation"]["sparsity"]),
        np.mean(metrics_clean["RISE"]["sparsity"]),
    ],
    "Entropy_Clean": [
        np.mean(metrics_clean["GradCAM"]["entropy"]),
        np.mean(metrics_clean["Ablation"]["entropy"]),
        np.mean(metrics_clean["RISE"]["entropy"]),
    ],
})
df_clean.to_csv("/content/table_clean_metrics.csv", index=False)

df_final = pd.DataFrame({
    "Method": ["GradCAM++", "AblationCAM", "RISE (Ours)"],
    "Faithfulness (Raw)": df_raw["Faithfulness_Correlation"],
    "Sparsity (Clean)": df_clean["Sparsity_Gini_Clean"],
    "Complexity (Clean)": df_clean["Entropy_Clean"],
})
df_final.to_csv("/content/FINAL_PUBLICATION_TABLE.csv", index=False)

# ---------------------- Saving PNG panels----------------------
def save_panel(img_list, title, filename):
    fig, ax = plt.subplots(3, 3, figsize=(12,12))
    fig.suptitle(title, fontsize=20, weight='bold')
    row_titles = ["NORMAL","BACTERIA","VIRUS"]
    for i, axis in enumerate(ax.flat):
        axis.imshow(img_list[i])
        axis.set_title(labels[i], fontsize=10)
        if i % 3 == 0:
            axis.set_ylabel(row_titles[i//3], fontsize=14, weight='bold')
        axis.set_xticks([]); axis.set_yticks([])
    plt.tight_layout()
    plt.savefig(f"/content/{filename}.png", dpi=300, bbox_inches='tight')
    plt.close()

save_panel(rise_images, "RISE", "Figure_1_RISE")
save_panel(grad_images, "GradCAM++", "Figure_2_GradCAM")
save_panel(ablation_images, "AblationCAM", "Figure_3_Ablation")

# ---------------------- ZIP & DOWNLOAD ----------------------
print("Creating ZIP...")
os.system("zip -r /content/Complete_Research_Results.zip /content/*.csv /content/*.png")

print("Starting download...")
files.download("/content/Complete_Research_Results.zip")

# ---------------------- Identifying high-confidence errors ----------------------
print("Scanning for high-confidence mistakes...")
fp_paths, fn_paths = [], []

for i, (img, label) in enumerate(test_ds):
    img_tensor = img.unsqueeze(0).to(device)
    with torch.no_grad():
        score = torch.sigmoid(model(img_tensor)).item()

    path = test_ds.samples[i][0]
    if label == 0 and score > 0.90 and len(fp_paths) < 2:
        fp_paths.append(path)
    if label == 1 and score < 0.10 and len(fn_paths) < 2:
        fn_paths.append(path)

error_paths = fp_paths + fn_paths
print("Errors detected:", len(error_paths))

for p in error_paths:
    if "NORMAL" in p:
        lbl = "False Positive (Normal predicted as Pneumonia)"
    else:
        lbl = "False Negative (Pneumonia predicted as Normal)"
    show_rise_clean(p, lbl)

print("Block 3 complete.")
